In [1]:
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from pyiron_atomistics import Project

2022-11-11 10:50:49,228 - pyiron_log - WARNING - Configuration file may be read by others - check permissions to secure credential information!
/home/menon/miniconda3/envs/pyiron-atom-dev/lib/python3.10/site-packages/paramiko-2.10.3-py3.10.egg/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
/home/menon/miniconda3/envs/pyiron-atom-dev/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: NOT-A-GIT-REPOSITORY is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
ev_to_j = sc.physical_constants["electron volt-joule relationship"][0]
Av =  sc.physical_constants["Avogadro constant"][0]
A_to_m = 1E-10
kB = sc.physical_constants["Boltzmann constant"][0]

In [3]:
projectname = "cp_t2"
element = "Al"
repetitions = (4,4,4)
temperature = 293
pressure = 0
potential = '2008--Mendelev-M-I--Al--LAMMPS--ipr1'
atomic_mass = 26.98

In [4]:
def create_project(projectname = None):
    pr = Project(projectname)
    return projectname

In [5]:
def create_structure(projectname = None, element=None, repetitions=(4,4,4)):
    pr = Project(projectname)
    structure = pr.create.structure.bulk(element, cubic=True).repeat(repetitions)
    natoms = len(structure)
    outfile = "initial_structure.data"
    structure.write(outfile, format="vasp")
    return outfile, natoms

In [6]:
def equilibriate_structure(projectname=None, structure=None, temperature=None, 
                          pressure=None, potential=None):
    pr = Project(projectname)
    job = pr.create.job.Lammps("job1", delete_existing_job=True,
                              delete_aborted_job=True)
    structure = pr.create.structure.read(structure, format="vasp")
    job.structure = structure
    job.potential = potential
    job.calc_md(temperature=temperature, 
                pressure=pressure, 
                n_ionic_steps=10000, 
                initial_temperature=temperature)
    job.run()
    eq_struct = job.get_structure()
    outfile = "equilibrium_structure.data"
    eq_struct.write(outfile, format="vasp")
    return outfile    

In [7]:
def get_energy_volume(projectname=None, structure=None, temperature=None, 
                          pressure=None, potential=None):
    pr = Project(projectname)
    job = pr.create.job.Lammps("job2", delete_existing_job=True,
                              delete_aborted_job=True)
    structure = pr.create.structure.read(structure, format="vasp")
    job.structure = structure
    job.potential = potential
    job.calc_md(temperature=temperature, 
                pressure=pressure, 
                n_ionic_steps=100000, 
                initial_temperature=temperature,
                n_print=100)
    job.run()
    energy_total = job.output.energy_tot
    volume = job.output.volume
    return energy_total, volume

In [8]:
def calculate_cp(energy=None, no_atoms=None, 
                 temperature=None, atomic_mass=None):
    emean = np.mean(energy)
    efluct = energy - emean
    efluctsq = (efluct*ev_to_j)**2
    cp = np.mean(efluctsq)/(kB*temperature*temperature)
    w = (no_atoms/Av)*1E3*atomic_mass
    cp = (cp/w)*1000
    return cp

In [9]:
def calculate_ap(energy=None, volume=None, temperature=None):
    emean = np.mean(energy)
    efluct = energy - emean
    crossfluct = (efluct*ev_to_j)*(volume - np.mean(volume))
    ap = np.mean(crossfluct)/(kB*temperature*temperature*np.mean(volume))
    return ap

In [10]:
projectname = create_project(projectname)
projectname

'cp_t2'

In [11]:
structure, no_atoms = create_structure(projectname=projectname, element=element,
                            repetitions=repetitions)
structure, no_atoms

('initial_structure.data', 256)

In [12]:
equil_structure = equilibriate_structure(projectname=projectname, 
                          structure=structure, 
                          temperature=temperature, 
                          pressure=pressure, 
                          potential=potential)
equil_structure

The job job1 was saved and received the ID: 353


'equilibrium_structure.data'

In [13]:
energy, volume = get_energy_volume(projectname=projectname, structure=equil_structure, 
                          temperature=temperature, 
                          pressure=pressure, potential=potential)

The job job2 was saved and received the ID: 354


In [14]:
cp = calculate_cp(energy=energy, no_atoms=no_atoms, 
                 temperature=temperature, atomic_mass=atomic_mass)
cp

0.9360438833189695

In [15]:
ap = calculate_ap(energy=energy, volume=volume, temperature=temperature)
ap

6.292018112751299e-05